In [1]:
import torch
import numpy as np
from toponetx import SimplicialComplex
from topomodelx.nn.simplicial.hsn_layer import HSNLayer

# Create signal and domain

The first step is to define the topological domain on which the TNN will operate, as well as the neighborhod structures characterizing this domain. We will only define th eneighborhood matrices that we plan on using.

Here, we build a simple simplicial complex domain.

In [2]:
edge_set = [[1, 2], [1, 3]]

domain = SimplicialComplex(edge_set)
domain.simplices

SimplexView([(1,), (2,), (3,), (1, 2), (1, 3)])

Now we retrieve the boundary matrix (or incidence matrix) associated to the faces of this complex.

In [3]:
incidence_1 = domain.incidence_matrix(rank=1)
adjacency_0 = domain.adjacency_matrix(rank=0)

print("incidence_1\n", incidence_1.todense())
print("adjacency_0\n", adjacency_0.todense())


incidence_1
 [[-1. -1.]
 [ 1.  0.]
 [ 0.  1.]]
adjacency_0
 [[0. 1. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


For each rank, the signal on this domain will look like a matrix with shape n_cells_of_rank_r x in_channels, where in_channels is the dimension of each cell's feature. In a a heterogenous domain, in_channels will vary by rank.

In [4]:
x_nodes = torch.tensor([[1.0, 1.0], [2.0, 2.0], [1.0, 1.0]])
channels_nodes = x_nodes.shape[-1]

In [5]:
incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
adjacency_0 = torch.from_numpy(adjacency_0.todense()).to_sparse()

# Create Neural Network

Stack layers.

In [6]:
class HSN(torch.nn.Module):
    def __init__(
            self,
            channels,
            incidence_matrix_1,
            adjacency_matrix_0,
            initialization="xavier_uniform",
            n_layers=2):
        super().__init__()
        modules = []
        for _ in range(n_layers):
            print("append layer")
            modules.append(
                HSNLayer(
                    channels=channels,
                    incidence_matrix_1=incidence_matrix_1,
                    adjacency_matrix_0=adjacency_matrix_0,
                    initialization=initialization,
        ))
        self.sequential = torch.nn.Sequential(*modules)
        self.linear = torch.nn.Linear(channels, 1)

    def forward(self, x_faces):
        x_faces = self.sequential(x_faces)
        return self.linear(x_faces)

In [7]:
model = HSN(channels=channels_nodes, incidence_matrix_1=incidence_1, adjacency_matrix_0=adjacency_0, n_layers=2)
nodes_gt_labels = torch.Tensor([[0], [1], [1]])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(5):
    optimizer.zero_grad()
    print(x_nodes, x_nodes.shape)
    nodes_pred_labels = model(x_nodes)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(nodes_pred_labels, nodes_gt_labels)
    loss.backward()
    optimizer.step()

append layer
build messages
append layer
build messages
tensor([[1., 1.],
        [2., 2.],
        [1., 1.]]) torch.Size([3, 2])
forward HSNLayer
weighted_x torch.Size([3, 2])
neighborhood torch.Size([3, 3])
weighted_x torch.Size([3, 2])
neighborhood torch.Size([2, 3])


TypeError: '_Merge' object is not iterable